# Predict hard drive failures using a Gaussian naive bayes classifier to reach ~95% accuracy!

Hard drives feature SMART stats that monitor indicators of hard drive status & reliability. Out of over 250 SMART stats, we will aim to choose 5-7 stats that predict hard drive failure. Inspired by [Backblaze's research](https://www.backblaze.com/blog/what-smart-stats-indicate-hard-drive-failures/) we will analyze SMART 5, 12, 184, 187, 188, 197, and 198 stats. 

## Feature selection: How do we choose these SMART features?


| Attribute  | Description                   |
|------------|-------------------------------|
| SMART 5    | Reallocated Sectors Count     |
| SMART 12   | Power Cycle Count             |            
| SMART 187  | Reported Uncorrectable Errors |
| SMART 188  | Command Timeout               |
| SMART 197  | Current Pending Sector Count  |
| SMART 198  | Uncorrectable Sector Count    |

Only choosing features that correlate with hard drive failure is crucial to building an accurate classifier. Features like SMART 5, 197, and 198 record the number of unused, unstable, or bad sectors in a hard drive, which affects the usability and lifespan of a drive. Features that count errors in reads of data such as SMART 187 also indicate problems in features critical to drive operation. SMART 188 measures the number of failed operations due to timeout, indicating perhaps critical problems with the power supply or data cable. Finally, SMART 12 refers to the remaining estimated power cycles of the hard drive, a feature estimated by the manufacturer.

Let's import our modules & data first, then reindex the dataframes to be in the same order as the features array.

In [215]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
features = ['smart_5_raw', 'smart_12_raw', 'smart_187_raw', 'smart_188_raw',
            'smart_197_raw', 'smart_198_raw', 'failure']
train_data = pd.read_csv("jan_feb_backblaze_train.csv").reindex(columns=features)
test_data = pd.read_csv("nov_dec_backblaze_test.csv").reindex(columns=features)

# Shuffle rows
train_data = train_data.sample(frac=1)
test_data = test_data.sample(frac=1)
train_data.head()

,smart_5_raw,smart_12_raw,smart_187_raw,smart_188_raw,smart_197_raw,smart_198_raw,failure
1104,0,7,0.0,NaN,0,NaN,0
1995,0,12,NaN,NaN,0,NaN,0
841,0,1,0.0,NaN,0,NaN,0
2144,32,4,12.0,NaN,0,NaN,1
913,0,7,0.0,NaN,0,NaN,0


## Analyzing & Preprocessing the Data

Using [harddrive data](https://www.backblaze.com/b2/hard-drive-test-data.html) from Backblaze that I've preprocessed to include an approximately 1:10 ratio of failed to working drives, let's take a look at the data. In the printed data above, there seems to be missing values in smart_187_raw.

In [207]:
print train_data.smart_187_raw.value_counts(dropna=False)
print test_data.smart_187_raw.value_counts(dropna=False)

 0.0     1357
NaN       798
 1.0       14
 2.0        9
 12.0       4
 4.0        4
 7.0        3
 6.0        3
 3.0        3
 36.0       2
 18.0       2
 8.0        2
 31.0       1
 30.0       1
 21.0       1
 9.0        1
 24.0       1
 39.0       1
Name: smart_187_raw, dtype: int64
 0.0     1349
NaN       825
 2.0       13
 1.0        8
 6.0        7
 12.0       4
 3.0        4
 4.0        3
 18.0       1
 11.0       1
 33.0       1
 78.0       1
 24.0       1
 17.0       1
 5.0        1
 7.0        1
 9.0        1
 27.0       1
 8.0        1
Name: smart_187_raw, dtype: int64


### Handling missing data
There seems to be approx. 800 missing values in those 2 rows. Filling the missing values with -1 will help our naive bayes model run without corrupting the available data.


In [208]:
train_data = train_data.fillna(value=-1)
test_data = test_data.fillna(value=-1)

## Why Gaussian Naive Bayes?

[Naive bayes](https://www.analyticsvidhya.com/blog/2015/09/naive-bayes-explained/) is a classifer based on Bayes' theorem. It "naively" assumes independence between features to calculate the probability of an instance belonging to a certain class. 

Because the SMART stats are relatively independent of each other, naive bayes is useful for classifying hard drive failures. Since we have a multitude of data from Backblaze, we're assuming a Gaussian distribution of data. Gaussian naive bayes computes the probability of belonging to each class (1 or 0; e.g. failed or working drive) based on the probability density function:
![alt](https://wikimedia.org/api/rest_v1/media/math/render/svg/acae0ab7740006874d2c7fd77eb5de61db3586c5)
The probability distribution of v given a class c is then calculated by:
![alt](https://wikimedia.org/api/rest_v1/media/math/render/svg/12ac511145223037a1378689333fe04c621845d4)

![alt](https://www.researchgate.net/profile/Yune_Lee/publication/255695722/figure/fig1/AS:297967207632900@1448052327024/Figure-1-Illustration-of-how-a-Gaussian-Naive-Bayes-GNB-classifier-works-For-each.png)
<p>After calculating the probability p(x | C) for some class c, the distance from the class mean divided by the standard deviation of that class is measured (also known as z-score). Gaussian Naive Bayes will classify point x as belonging in whichever class the point x is closest to (with the lowest z-score). </p>
<p>In the picture above, point X has a z-score (distance) closer to the mean of class A when observed as if it belonged in both classes A & B. Therefore it will classify as belonging to class A.</p>

## Training & Testing Gaussian Naive Bayes 

In [210]:
"""Fit the sklearn gaussian naive bayes to the training and testing data"""
gnb = GaussianNB()

# Training data
train_ds = train_data.values
train_target = train_ds[:, -1]
train_ds = train_ds[:, 0:-1]

# Testing data
test_ds = test_data.values
test_target = test_ds[:, -1]
test_ds = test_ds[:, 0:-1]


gnb = gnb.fit(train_ds, train_target)
y_pred = gnb.predict(test_ds)

# Count number of predictions that missed predicting an actual hard drive failure (predicted 0 when actually was 1)
missed_failed_pred = sum([test_target[i] == 1 and y_pred[i] == 0 for i in range(len(test_target))])
incorrect = (test_target != y_pred).sum()
correct = (test_target == y_pred).sum()
total = test_ds.shape[0]
print "Num mislabeled pts out of total %d points: %d" % (total, incorrect)
print "Num missed failed hard drive predictions: %s; %s%% out of total" % (missed_failed_pred, 100.0 * float(missed_failed_pred) / total)
print "Percent accuracy: %s%%" % (str(100.0 * gnb.score(test_ds, test_target)))

Num mislabeled pts out of total 2224 points: 127
Num missed failed hard drive predictions: 116; 5.21582733813% out of total
Percent accuracy: 94.2895683453%


### Analyzing GNB's Predictions
<p>With Gaussian Naive Bayes we achieved an accuracy close to 95%. This makes our naive bayes model quite successful given its simple algorithm. Out of 128 mislabeled points, GNB labeled 13 points as false negatives, which gives it a 0.58% rate of false negatives (13/2224). Though the false negative rate is relatively low, GNB predicted false positives at a 5.17% rate, which is a pretty high rate if you consider the scale of modern day data centers. </p>
<p>For example, in a data center with 10,000 hard-drives, 500 drives might be flagged as false positives, causing a lot of extra overhead to determine manually which drives are failing.</p>
<p>How might we sort the predictions to prioritize which drives to check for failure?</p>
<p>According to the research at Backblaze, drives with a high number of SMART values greater than zero tend to report higher percentage of failures. To sort with high priority, we can apply a lambda function to sort drives by number of SMART values greater than 0. </p>
<p>Another approach would be to sort based on the sum of the SMART values. For all the SMART values we're analyzing except for SMART 12 (power cycle count), the values increment based on the number of errors. (Power cycle count starts at 100 and decreases based on estimated remaining HDD life)</p>
<p>Lastly, sorting by the power cycle count (SMART 12) could be a low priority method. Since power cycle counts are a more arbitrary estimation, it isn't the best indicator of HDD failure, but could be an interesting way to compare your results against the other 2 methods</p>


In [211]:
from collections import OrderedDict
# Populate dictionary with index and values of predicted failed drives
pred_failed_drives ={index: test_ds[index].tolist() for index, item in enumerate(y_pred.tolist()) if item == 1.0}

# Sort by number of SMART values > 0
high_preds = sorted(pred_failed_drives.items(), key=lambda t: sum(p>0 for p in t[1]), reverse=True)
print "High priority predicted drives:"
for i in range(5):
    print high_preds[i]
print "\n"

# Sort by sum of SMART values excluding SMART 12 (at index 1 of test_ds)
med_preds = sorted(pred_failed_drives.items(), key=lambda t: sum(t[1][0:1]+t[1][1:len(t[1])]), reverse=True)
print "Medium priority predicted drives:"
for i in range(5):
    print med_preds[i]
print "\n"

# Sort by power cycle count (SMART 12) (at index 1 of test_ds)
low_preds = sorted(pred_failed_drives.items(), key=lambda t: t[1][1])
print "Low priority predicted drives:"
for i in range(5):
    print low_preds[i]

High priority predicted drives:
(1395, [8.0, 2.0, 5.0, 24.0, 24.0])
(546, [1688.0, 24.0, 0.0, 40.0, 40.0])
(1625, [24.0, 9.0, 0.0, 24.0, 24.0])
(1471, [0.0, 6.0, 1.0, 8.0, 8.0])
(1158, [8.0, 4.0, 0.0, 80.0, 80.0])


Medium priority predicted drives:
(956, [472.0, 17.0, 0.0, 9736.0, 9736.0])
(1552, [3960.0, 0.0, 0.0, 32.0, 32.0])
(1222, [0.0, 0.0, 1.0, 960.0, 960.0])
(546, [1688.0, 24.0, 0.0, 40.0, 40.0])
(2176, [1416.0, 0.0, 0.0, 32.0, 32.0])


Low priority predicted drives:
(531, [102.0, -1.0, -1.0, 0.0, 0.0])
(2073, [560.0, -1.0, -1.0, 0.0, 0.0])
(549, [0.0, -1.0, -1.0, 16.0, 0.0])
(1578, [0.0, -1.0, -1.0, 8.0, 0.0])
(1072, [0.0, -1.0, -1.0, 7.0, 0.0])


## Experiment for fun!
<p>Find other SMART values or change which SMART values you use in the features array and retrain your naive bayes model. Try to achieve higher accuracy! Perhaps consider SMART 184 (end-to-end error), another indicator of critical problems.</p>
<p>Experiment with printing different ranges of values and finding similar hard drives between the 3 different methods of sorting the predictions.</p>
<p>For fun, let's implement another model of naive bayes, Bernoulli naive bayes, to classify the hard drives. Instead of computing the probabilities of raw values as Gaussian naive bayes does, Bernoulli naive bayes assumes binary values. Bernoulli naive bayes is most often used in text classification. Learn more about Bernoulli naive bayes [here](https://nlp.stanford.edu/IR-book/html/htmledition/the-bernoulli-model-1.html).

In [212]:
bnb = BernoulliNB(alpha=1.0, binarize=0.0)
bnb = bnb.fit(train_ds, train_target)
bnb_pred = bnb.predict(test_ds)

incorrect = (test_target != bnb_pred).sum()
correct = (test_target == bnb_pred).sum()
total = test_ds.shape[0]
print "Num mislabeled pts out of total %d points: %d" % (total, incorrect)
print "Percent accuracy: %s%%" % (str(100.0 * bnb.score(test_ds, test_target)))

Num mislabeled pts out of total 2224 points: 106
Percent accuracy: 95.2338129496%


# Conclusion
<p>Bernoulli naive bayes performed about 1% better than Gaussian naive bayes. This may indicate that a binary interpretation of SMART values is useful, meaning that an increase from 0 to any value in the chosen SMART values may be a strong enough indicator of failing drive. </p>
<p>For a simple binary classification, naive bayes is a useful algorithm to achieve quick results and pretty high accuracy! Naive bayes can serve as a useful baseline model for approaching classification projects. Building off of your analysis of the naive bayes predictions, it could be useful to build feed-forward neural networks (sklearn Multilayer Perceptron) or a decision tree classifier, and compare the results. </p>